In [18]:
import torch
import torch.nn as nn
from models import SelfAttention
from data_preprocessing import load_and_process_data
from train_test import train_and_test
import config
import torch.nn as nn
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [19]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
def load_and_process_data(is_scaler, path_xtrain, path_xtest, path_ytrain, path_ytest):

    x_train_reduction = pd.read_csv(path_xtrain)
    x_test_reduction = pd.read_csv(path_xtest)
    y_train = pd.read_csv(path_ytrain)
    y_test = pd.read_csv(path_ytest)


    x_train_reduction = x_train_reduction.values.astype(float)
    x_test_reduction = x_test_reduction.values.astype(float)
    y_train = y_train.values.astype(float)
    y_test = y_test.values.astype(float)


    # 2.将数据进行标准化
    if is_scaler:
        scaler = StandardScaler()
        scaler_model = StandardScaler()
        scaler.fit(x_train_reduction)
        x_train_reduction = scaler.transform(x_train_reduction)
        x_test_reduction = scaler.transform(x_test_reduction)

        scaler_model.fit(y_train)
        y_train = scaler_model.transform(y_train)
        y_test = scaler_model.transform(y_test)

    x_train_tensor = torch.from_numpy(x_train_reduction).to(torch.float32).to(DEVICE)
    y_train_tensor = torch.from_numpy(y_train).to(torch.float32).to(DEVICE)
    x_test_tensor = torch.from_numpy(x_test_reduction).to(torch.float32).to(DEVICE)
    y_test_tensor = torch.from_numpy(y_test).to(torch.float32).to(DEVICE)

    train_data = TensorDataset(x_train_tensor, y_train_tensor)
    test_data = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size,
                                               True)

    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size,
                                              False)

    return x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor

In [22]:
path_xtrain="xtrain_ori.csv"
path_xtest="xtest_ori.csv"
path_ytrain="ytrain_ori.csv"
path_ytest="ytest_ori.csv"

In [ ]:
x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor = load_and_process_data(
    is_scaler=config.is_scaler,
    path_xtrain = path_xtrain,
    path_xtest = path_xtest,
    path_ytrain = path_ytrain,
    path_ytest = path_ytest)

In [9]:
DEVICE = config.DEVICE
batch_size = config.batch_size
hidden_dim = config.hidden_dim
output_dim = config.output_dim
num_attention_heads = config.num_attention_heads
kernel_size = config.kernel_size
LR = config.LR
epochs = config.epochs
hidden_dropout_prob = config.hidden_dropout_prob
attention_probs_dropout_prob = config.attention_probs_dropout_prob
is_scaler = config.is_scaler

In [13]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False)

In [14]:
model = SelfAttention(
    num_attention_heads=config.num_attention_heads,
    input_size=x_train_tensor.shape[1],
    hidden_size=config.hidden_dim,
    output_dim=config.output_dim,
    kernel_size=config.kernel_size,
    hidden_dropout_prob=config.hidden_dropout_prob,
    attention_probs_dropout_prob=config.attention_probs_dropout_prob
).to(DEVICE)

In [15]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)

In [16]:
import torch
from tqdm import tqdm

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_train = 0
    correct_train = 0
    running_loss = 0.0

    for data in tqdm(train_loader, desc="Training"):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze().long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels.squeeze().long()).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_train / total_train

    return train_loss, train_accuracy

def test(model, test_loader, criterion, device):
    model.eval()
    total_test = 0
    correct_test = 0
    running_loss = 0.0

    with torch.no_grad():
        for data in tqdm(test_loader, desc="Testing"):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze().long())

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
    
            total_test += labels.size(0)
        
            correct_test += (predicted == labels.squeeze().long()).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct_test / total_test

    return test_loss, test_accuracy

def train_and_test(model, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_acc = 0.0

    for epoch in range(epochs):
        
        train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)
        
        print(f'Epoch {epoch + 1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')


        print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

        if test_accuracy > best_acc:
            best_acc = test_accuracy
            print(f'Improved Test Accuracy: {best_acc:.4f} at epoch {epoch + 1}')
            # 保存模型
            torch.save(model.state_dict(), "best_model.pth")

    print('Training and testing complete.')

In [ ]:
train_and_test(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=loss_function,
    optimizer=optimizer,
    epochs=config.epochs,
    device=DEVICE
)